<style>div.container { width: 100% }</style>
<img style="float:left;  vertical-align:text-bottom;" height="65" width="172" src="../assets/holoviz-logo-unstacked.svg" />
<div style="float:right; vertical-align:text-bottom;"><h2>Tutorial 3: Adding Interactivity to Panel</h2></div>

<div class="alert alert-warning" role="alert"> <strong>WORK IN PROGRESS:</strong> We are in the progress of updating these materials in anticipation of a tutorial at the 2019 SciPy conference. Work will be complete by the morning of July 8th 2019. Check out <a href="https://github.com/pyviz/holoviz/tree/v0.1.1">this tag</a> to access the materials as they were before these changes started. For the latest version of the tutorial, visit <a href="https://holoviz.org/tutorial">holoviz.org</a>.
</div>


In the previous section we learned the very basics of working with Panel. Specifically we looked at the different types of components, how to dynamically update them and how to serve a Panel application or dashboard. However to start building actual apps with Panel we need to be able to add interactivity by linking different components together. In this section we will learn how to link widgets to outputs to start building some simple interactive applications.

In [ ]:
import panel as pn

pn.extension()

## Widgets

One of the most common patterns when building an application is linking a widget to some output and updating it in response. As we briefly mentioned in the last section, a widget is an input control which allows a user to change a ``value`` using some graphical UI. A simple example is a slider:

In [ ]:
slider = pn.widgets.IntSlider(start=0, end=10)

slider

When we drag the slider the ``value`` parameter on the widget updates:

In [ ]:
slider.value

As a very simple example we can link the slider ``value`` to the ``object`` parameter on a ``Str`` pane:

In [ ]:
text = pn.pane.Str(str(slider.value))

slider.link(text, value='object')

text

Whenever the slider is dragged the text updates.

#### Exercise

Use tab-completion on the ``pn.widget`` namespace to discover some other widget, construct a string pane and then link the widget value to the pane ``object`` parameter. Finally display both the widget and the pane in a panel.

## Callbacks

In addition to directly linking parameters the more general approach is to write callbacks in response to changes in some parameter, e.g. the ``value`` of a widget. All parameters can be watched using the ``.param.watch`` API, which will call the provided callback with an event object, which makes the changed value available.

## Declaring interactive components

There are a variety of ways of linking different components in Panel, with linking just being one very simple example. Another approach is to write explicit callbacks, but this can quickly lead to very messy code. A different approach is to express dependencies between the parameters of one object and declare those as inputs to a function.

As a very simple example we will declare a ``TextInput`` widget and then write a function, which uses Markdown syntax to convert the text into a title. Using the ``pn.depends`` decorator we can then declare that this function depends on the ``value`` of the widget. Finally we lay out the widget and the function:

In [ ]:
text_input = pn.widgets.TextInput()

@pn.depends(text_input.param.value)
def title_text(value):
    return '## ' + value

pn.Row(text_input, title_text)

When entering some text into the widget (and pressing enter) we will see that it appears as a title. In this way we can easily declare dynamic components which depend on the value of a widget.

#### Exercise

Declare two ``Spinner`` widgets with an initial value of 1, then declare a function which depends on the values of both widgets and adds them together. Finally lay out the two widgets and the function in a Panel:

In [ ]:
a = pn.widgets.Spinner(value=1, width=60)
b = pn.widgets.Spinner(value=1, width=60)

@pn.depends(a.param.value, b.param.value)
def adder(a, b):
    return a + b

pn.Row(a, '+', b, '=', adder)

<details><summary>Solution</summary><br>

a = pn.widgets.Spinner(value=1, width=60)
b = pn.widgets.Spinner(value=1, width=60)

@pn.depends(a.param.value, b.param.value)
def adder(a, b):
    return a + b

pn.Row(a, '+', b, '=', adder)

</details>

## Exercise: Building a dashboard

In this section we will build a dashboard, we will define the different components and as an exercise you will assemble them into the overall dashboard.

In [ ]:
import panel as pn

pn.extension('katex')

#### The data

Throughout this tutorial we will be working with one core dataset, a collection of earthquakes recorded between 2000-2018 provided by the US Geological Survey (USGS). The data is provided as a parquet file as part of the tutorial and we will load it using dask and persist it. We will return to this later, for now we will focus on building a dashboard and you don't know any of the details about the dataset or the dask or pandas API.

In [ ]:
import dask.dataframe as dd

df = dd.read_parquet('../data/earthquakes.parq')

df = df[~df.mag.isna()].persist()

#### The logo

The first component of the dashboard is an image of the US Geological Survey logo, start by declaring a pane containing the logo and assign it to the ``logo`` variable.

In [ ]:
logo_url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/c/cd/USGS_logo.svg/2000px-USGS_logo.svg.png'

## Define a panel component containing the logo


## Display it


#### Richter scale equation

Next we will create a component to display the equation for the Richter scale definition. Declare the appropriate pane and assign it to the ``equation`` variable.

In [ ]:
equation_string = '$M_L = log_{10}A - log_{10} A_0(\delta)$'

## Define a panel component containing the equation (Hint: Use the LaTeX pane)


## Display it


#### Widgets

In [ ]:
year_slider = pn.widgets.IntSlider(name='Year', start=2000, end=2018, value=2000)

year_slider

#### Computing statistics

In [ ]:
## Add a depends decorator to the function to link it to the widget
@pn.depends(year_slider.param.value)
def yearly_stats(year):
    return df[df.time.dt.year == year][['mag', 'depth', 'rms']].describe().compute()

## Display the function


#### List the strongest earthquakes

In [ ]:
## Add a depends decorator to the function to link it to the widget
@pn.depends(year_slider.param.value)
def strongest_earthquakes(year):
    year_df = df[df.time.dt.year == year].compute()
    return year_df.sort_values('mag', ascending=False).iloc[:5][['time', 'place', 'mag']].reset_index(drop=True)

## Display the function


### Composing the dashboard

Now that we have defined all the different components, it is time to lay them out into the overall dashboard.

Arrange the following components into a dashboard using the ``Row`` and ``Column`` panels:

* ``logo``
* ``equation``
* ``year_slider``
* ``yearly_stats``
* ``strongest_earthquakes``

In [ ]:
pn.Row(
    pn.Column(pn.panel(logo_url, width=300), year_slider, pn.panel('## The Richter scale'), pn.pane.LaTeX(equation_string)),
    pn.Column('### Statistics', pn.depends(year_slider.param.value)(yearly_stats)),
    pn.Column('### Strongest Earthquakes', pn.depends(year_slider.param.value)(strongest_earthquakes))
)